merge the following CSV files:
1. file with ID(int) and SCH(float)
2. file with ID(int), Level(char)
<br>
into one with the header, “ID​, SCH, Level​.”

In [1]:
import random,string
import pandas as pd
import numpy as np

In [2]:
pd.DataFrame({'ID':np.arange(0,7),'SCH':np.random.rand(7)}).to_csv('file1.csv')
pd.DataFrame({'ID':np.arange(0,7),'Level':[random.choice(string.ascii_uppercase) for i in range(7)]}).to_csv('file2.csv')

In [3]:
file1 = pd.read_csv('file1.csv',index_col=0)
file2 = pd.read_csv('file2.csv',index_col=0)
file1.head()

,ID,SCH
0,0,0.253378
1,1,0.759996
2,2,0.046530
3,3,0.871747
4,4,0.154332


In [4]:
file2.head()

,ID,Level
0,0,N
1,1,R
2,2,P
3,3,J
4,4,V


In [5]:
file1.merge(file2,how='inner',on='ID')

,ID,SCH,Level
0,0,0.253378,N
1,1,0.759996,R
2,2,0.046530,P
3,3,0.871747,J
4,4,0.154332,V
5,5,0.406715,Y
6,6,0.443318,D


Convert Table1 into the format of Table 2 and then combine the tables into a single file

In [6]:
table1 = pd.read_csv('table1.csv')
table2 = pd.read_csv('table2.csv')
table1

,Semester,Sam ID,Method,First Name,Last Name
0,Fall 2010,459044,I,Varun,Penmetsa
1,Spring 2011,345682,I,Hari,Nandamuri
2,Summer I 2011,347687,R,Sai,Yalamanchili
3,Summer II 2011,236875,C,Satish,Ravella
4,Fall 2009,345792,D,Vishnu,Vasireddy
5,Spring 2009,347593,A,Mahi,Chaganti
6,Fall 2008,345545,I,Vasu,Achanta


In [7]:
table2

,Semester,Sam ID,Method,Full Name
0,201080,459044,Internet,Varun Penmetsa
1,201020,456788,Record,Vamsi Nuthakki
2,201040,467899,Class,Krish Yattikonda
3,201060,475998,Data,Divya Bhoga
4,201180,347687,All,Teja Yundavally
5,201120,345682,Internet,Hari Nandamuri
6,201140,347687,Record,Sai Yalamanchili
7,201160,236875,Class,Satish Ravella


To convert Table 1 into the format of Table 2

In [8]:
# understanding common records
table1.merge(table2,on='Sam ID')

,Semester_x,Sam ID,Method_x,First Name,Last Name,Semester_y,Method_y,Full Name
0,Fall 2010,459044,I,Varun,Penmetsa,201080,Internet,Varun Penmetsa
1,Spring 2011,345682,I,Hari,Nandamuri,201120,Internet,Hari Nandamuri
2,Summer I 2011,347687,R,Sai,Yalamanchili,201180,All,Teja Yundavally
3,Summer I 2011,347687,R,Sai,Yalamanchili,201140,Record,Sai Yalamanchili
4,Summer II 2011,236875,C,Satish,Ravella,201160,Class,Satish Ravella


Sam ID is not the primary key in table 2 as it is repeated for two students, namely, Teja and Sai

In [9]:
# Semester mapping -  manual investigation reveals simple mapping

In [10]:
# for each entry of semester in table1, strip the year and return unique entries
set(table1.Semester.apply(lambda x :' '.join(str.split(x)[:-1])))

{'Fall', 'Spring', 'Summer I', 'Summer II'}

In [11]:
#create mapping
d = dict({'Spring' : 20,
          'Summer I' : 40,
          'Summer II' : 60,
          'Fall' : 80})
d

{'Fall': 80, 'Spring': 20, 'Summer I': 40, 'Summer II': 60}

In [12]:
table1_new = table1.copy()

In [13]:
# function to transform 'Semester'
def sem(x):
    tokens = str.split(x)
    return ''.join([tokens[-1],str(d[' '.join(tokens[:-1])])]) # take the year and join it with the semester mapping

In [14]:
table1_new['Semester_new'] = table1_new.Semester.apply(lambda x: sem(x))
table1_new

,Semester,Sam ID,Method,First Name,Last Name,Semester_new
0,Fall 2010,459044,I,Varun,Penmetsa,201080
1,Spring 2011,345682,I,Hari,Nandamuri,201120
2,Summer I 2011,347687,R,Sai,Yalamanchili,201140
3,Summer II 2011,236875,C,Satish,Ravella,201160
4,Fall 2009,345792,D,Vishnu,Vasireddy,200980
5,Spring 2009,347593,A,Mahi,Chaganti,200920
6,Fall 2008,345545,I,Vasu,Achanta,200880


In [15]:
# Method mapping
a = sorted(set(table1_new.Method))
b = sorted(set(table2.Method))
met_dict = dict(zip(a,b))
met_dict

{'A': 'All', 'C': 'Class', 'D': 'Data', 'I': 'Internet', 'R': 'Record'}

In [16]:
table1_new['Method_new'] = table1_new.Method.apply(lambda x : met_dict[x])
table1_new

,Semester,Sam ID,Method,First Name,Last Name,Semester_new,Method_new
0,Fall 2010,459044,I,Varun,Penmetsa,201080,Internet
1,Spring 2011,345682,I,Hari,Nandamuri,201120,Internet
2,Summer I 2011,347687,R,Sai,Yalamanchili,201140,Record
3,Summer II 2011,236875,C,Satish,Ravella,201160,Class
4,Fall 2009,345792,D,Vishnu,Vasireddy,200980,Data
5,Spring 2009,347593,A,Mahi,Chaganti,200920,All
6,Fall 2008,345545,I,Vasu,Achanta,200880,Internet


In [17]:
#Full name
for i,r in table1_new.iterrows():
    table1_new.loc[i,'Full Name'] = ' '.join([r['First Name'],r['Last Name']])
table1_new

,Semester,Sam ID,Method,First Name,Last Name,Semester_new,Method_new,Full Name
0,Fall 2010,459044,I,Varun,Penmetsa,201080,Internet,Varun Penmetsa
1,Spring 2011,345682,I,Hari,Nandamuri,201120,Internet,Hari Nandamuri
2,Summer I 2011,347687,R,Sai,Yalamanchili,201140,Record,Sai Yalamanchili
3,Summer II 2011,236875,C,Satish,Ravella,201160,Class,Satish Ravella
4,Fall 2009,345792,D,Vishnu,Vasireddy,200980,Data,Vishnu Vasireddy
5,Spring 2009,347593,A,Mahi,Chaganti,200920,All,Mahi Chaganti
6,Fall 2008,345545,I,Vasu,Achanta,200880,Internet,Vasu Achanta


Combine the tables into a single file

In [18]:
# since the question is to just combine I'm using a full outer join
table1.merge(table2,how='outer')

,Semester,Sam ID,Method,First Name,Last Name,Full Name
0,Fall 2010,459044,I,Varun,Penmetsa,NaN
1,Spring 2011,345682,I,Hari,Nandamuri,NaN
2,Summer I 2011,347687,R,Sai,Yalamanchili,NaN
3,Summer II 2011,236875,C,Satish,Ravella,NaN
4,Fall 2009,345792,D,Vishnu,Vasireddy,NaN
5,Spring 2009,347593,A,Mahi,Chaganti,NaN
6,Fall 2008,345545,I,Vasu,Achanta,NaN
7,201080,459044,Internet,NaN,NaN,Varun Penmetsa
8,201020,456788,Record,NaN,NaN,Vamsi Nuthakki
9,201040,467899,Class,NaN,NaN,Krish Yattikonda
